<a href="https://colab.research.google.com/github/rohitrmd/replicate-introduction/blob/main/Introduction_to_replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction to [replicate](https://replicate.com/)

While looking at different platforms to run stable diffusion models, I came across [replicate](https://replicate.com/).  While stability.ai does provide you API to do inference against stable diffusion models, replicate allows you to  run your own custom models along with open source models. Also it has much bigger community of  [open source models](https://replicate.com/explore).

In this tutorial we will look at how we can run simple image, audio and video generation models using replicate.  ![Screenshot 2024-01-28 at 11.18.08 AM.png])

### Setup

1.   First you need to go to https://replicate.com/ and sign in using your github account.
2.   Then create your API token on [tokens page](https://replicate.com/account/api-tokens).
3.   Copy this token to secrets section with key as REPLICATE_API_TOKEN and value as token's value
4.   Setup this token value as environment variable as showed below





In [1]:
import os
from google.colab import userdata
api_token = userdata.get('REPLICATE_API_TOKEN')
os.environ["REPLICATE_API_TOKEN"] = api_token

### Installation
We need to install replicate python API client to access their environment and also have access to their abstraction over ML models.

In [2]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


### Run the model
Let's run stable diffusion SDXL model for image generation. We use replicate.run method to provide prompt to the model.

In [3]:
import replicate

output = replicate.run(
  "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
  input={"prompt": "a sunset on the beach"}
)

from IPython.display import Image
Image(url=output[0])

### Running SDXL video generation model
Ever since stability ai released stable video diffusion, I wanted to try that out. replicate makes it extremely easy to run this model and results are spectacular. Keep in mind this model is not ready yet for commercial use.

In [33]:
from urllib.request import urlretrieve

model = replicate.models.get("stability-ai/stable-video-diffusion")
version = model.versions.get("3f0457e4619daac51203dedb472816fd4af51f3149fa7a9e0b5ffcf1b8172438")

prediction = replicate.predictions.create(
    version=version,
    input={
      "cond_aug": 0.02,
      "decoding_t": 7,
      "input_image": "https://raw.githubusercontent.com/rohitrmd/replicate-introduction/main/astronaut.jpg",
      "video_length": "14_frames_with_svd",
      "sizing_strategy": "maintain_aspect_ratio",
      "motion_bucket_id": 127,
      "frames_per_second": 6
    })
prediction.wait()



### Tracking prediction status
As image/audio/video generation models take long time to generate output, replicate provides easy way to track progress of the inference. Status can have values as `started`, `processing`, `succeeded` or `failed`. Let's verify status of our prediction.

In [34]:
prediction.status

'succeeded'

### View the video output

In [35]:
urlretrieve(prediction.output, "/tmp/out.mp4")
from IPython.display import Video

Video(prediction.output)